In [4]:
from json2html import *
import json

In [3]:
# j = json.load(open('./vulnerabilities/2022/AVID-2022-V001.json'))
j = json.load(open('Untitled.txt'))

In [4]:
with open("sample.json", "w") as outfile:
    outfile.write(json.dumps(j))

In [16]:
json2html.convert(json = j, table_attributes="id=\"info-table\" class=\"table table-bordered table-hover\"")

'<table id="info-table" class="table table-bordered table-hover"><tr><th>vuln_id</th><td>AVID-2022-V001</td></tr><tr><th>metadata</th><td><table id="info-table" class="table table-bordered table-hover"><tr><th>class</th><td>LLM evaluation</td></tr><tr><th>taxonomy_version</th><td>0.1</td></tr></table></td></tr><tr><th>description</th><td><table id="info-table" class="table table-bordered table-hover"><tr><th>lang</th><td>eng</td></tr><tr><th>name</th><td>Gender Bias in Sentence Completion Tasks performed by bert-base-uncased</td></tr><tr><th>value</th><td>xyz xyz</td></tr></table></td></tr><tr><th>reports</th><td><table id="info-table" class="table table-bordered table-hover"><thead><tr><th>report_id</th><th>class</th><th>name</th></tr></thead><tbody><tr><td>AVID-2022-R0001</td><td>Measurement</td><td>Gender Bias in sentence completion Tasks performed by bert-base-uncased using the HONEST metric</td></tr><tr><td>AVID-2022-R0003</td><td>Measurement</td><td>Profession bias reinforcing ge

In [6]:
j = json.load(open('test.json'))

In [8]:
import jsonschema2md

parser = jsonschema2md.Parser(
    examples_as_yaml=False,
    show_examples="all",
)
md_lines = parser.parse_schema(j)
print(''.join(md_lines))

# Person

*JSON Schema for a person object.*

## Properties

- **`firstName`** *(string)*: The person's first name.
- **`lastName`** *(string)*: The person's last name.



In [12]:
import pyairtable
import os
from dotenv import load_dotenv
import json

load_dotenv()
api_key = os.getenv('AIRTABLE_API_KEY')
config = json.load(open('../connectors/config.json'))

In [80]:
# fetch data from airtable
all_data = pyairtable.Table(api_key, config['airtable']['base_id'], config['airtable']['table_name']).all()

In [39]:
def save_as_report(record):
    
    report = {}
    report['data_type'] = 'AVID'
    report['version'] = ''
    report['metadata'] = {
        'report_id' : ''
    }
    report['affects'] = {
        'developer': record['fields']['Developer of Artifact'],
        'deployer': record['fields']['Deployer of Artifact'],
        'artifact': [
            {
                'type': record['fields']['Artifact Type'],
                'name': record['fields']['Artifact Name']
            }
        ]
    }
    report['problemtype'] = {
        'class': '',
        'type': record['fields']['Report Type'].split(':')[0],
        'description': {
            'lang': 'eng',
            'value': record['fields']['Title']
        }
    }
    report['metrics'] = []
    report['references'] = record['fields']['References']
    report['description'] = {
        'lang': 'eng',
        'value': record['fields']['Description']
    }
    report['impact'] = {
        'avid': {
            'vuln_id': '',
            'risk_domain': record['fields']['Relevant SEP risk domains'],
            'sep_view': record['fields']['Relevant Ethics subcategories'],
            'lifecycle_view': record['fields']['Relevant stages of the AI lifecycle']
        }
    }
    report['credits'] = [
        {
            'lang': 'eng',
            'value': record['fields']['Submitter Name']
        }
    ]
    report['reported_date'] = record['createdTime'].split('T')[0]
    
    # save report
    output = open('../reports/dev/'+record['id']+'.json', 'w')
    json.dump(report, output, indent=4)
    output.close()

In [81]:
record = all_data[3].copy()

# fill essential empty entries
record_keys = list(record['fields'].keys())
strings = ['Submitter Organization'] + ['Relevant '+s+' subcategories' for s in ['Security','Ethics','Performance']]
for st in strings:
    if record_keys.count(st)==0:
        record[st] = ''

record

{'id': 'reciaOOxqMH4eCkOq',
 'createdTime': '2023-01-05T02:22:03.000Z',
 'fields': {'Developer of Artifact': 'OpenAI',
  'Deployer of Artifact': 'OpenAI',
  'Description': 'I asked ChatGPT to recommend papers on explainability, privacy, adversarial ML, etc. It did recommend me a list of papers but it linked wrong authors to the papers and some of the papers didn\'t even exist (maybe it just made up those paper titles). For example- when prompted to recommend papers on explainability, it said the paper "Explaining Explanations: An Overview of Interpretability of Machine Learning" is by Zach Lipton, which in fact, is written by Gilpin et al. and does not have Zach as an author. This potentially hints at misinformation. It made similar mistakes when asking for papers on privacy, interpretability, and adversarial ML. ',
  'Submission Date': '2023-01-04',
  'Artifact Type': 'Model',
  'Relevant SEP risk domains': ['Ethics'],
  'Report Type': 'Issue: qualitative evaluation based on a small s